In [3]:
#packages/modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import json
import os
import datetime
import plotly.express as px

In [4]:
#read in data
raw_data = pd.read_csv('P:\Georgia Tech\Curriculum\Fall 2022\CSE 6242 - Data and Visual Analytics\Project\Redfin Data\zip_code_market_tracker.tsv000', sep = '\t')

In [3]:
#see sample data
display(raw_data.head(5))

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2021-08-01,2021-10-31,90,zip code,2,20593,f,Zip Code: 48301,NaN,Michigan,...,0.080460,NaN,NaN,NaN,0.461538,0.253205,0.061538,"Warren, MI",47664,2022-10-16 14:38:47
1,2015-05-01,2015-07-31,90,zip code,2,29924,f,Zip Code: 70076,NaN,Louisiana,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"New Orleans, LA",35380,2022-10-16 14:38:47
2,2018-08-01,2018-10-31,90,zip code,2,5101,f,Zip Code: 14222,NaN,New York,...,-0.750000,NaN,NaN,NaN,NaN,NaN,NaN,"Buffalo, NY",15380,2022-10-16 14:38:47
3,2016-07-01,2016-09-30,90,zip code,2,13900,f,Zip Code: 32966,NaN,Florida,...,0.014394,NaN,NaN,NaN,0.333333,0.222222,0.222222,"Sebastian, FL",42680,2022-10-16 14:38:47
4,2020-09-01,2020-11-30,90,zip code,2,16709,f,Zip Code: 39180,NaN,Mississippi,...,-0.200000,NaN,NaN,NaN,0.000000,NaN,0.000000,"Vicksburg, MS",46980,2022-10-16 14:38:47


### How many rows?

In [4]:
print(len(raw_data))

6305970


### What features are important?

In [5]:
#what are the features/columns?
print(raw_data.columns)

#number of columns
print(len(raw_data.columns))


Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

# 0. Which features are important?
period_begin, period_end, period_duration, region, state_code, property_type, homes_sold, new_listings, median_sale_price, median_list_price, inventory 

### Definitions
inventory: total number of active listings on the last day of a given time period
new_listings: total number of homes with a listing added date during a given time period
months_of_supply: tells you how long it would take supply to be bought upt if no new homes came on the market

### Describe feature values

In [6]:
print(raw_data['region'].describe())
print()
print(raw_data['state_code'].describe())
print()
print(raw_data['property_type'].describe())
print()
print(raw_data['homes_sold'].describe())


count             6305970
unique              22266
top       Zip Code: 33407
freq                  635
Name: region, dtype: object

count     6305970
unique         48
top            CA
freq       628096
Name: state_code, dtype: object

count             6305970
unique                  5
top       All Residential
freq              2092546
Name: property_type, dtype: object

count    6.294143e+06
mean     3.628321e+01
std      5.718470e+01
min      1.000000e+00
25%      3.000000e+00
50%      1.100000e+01
75%      4.600000e+01
max      9.660000e+02
Name: homes_sold, dtype: float64


In [7]:
print(raw_data['new_listings'].describe())
print()
print(raw_data['median_sale_price'].describe())
print()
print(raw_data['median_list_price'].describe())

count    5.786043e+06
mean     4.580903e+01
std      6.786595e+01
min      1.000000e+00
25%      5.000000e+00
50%      1.600000e+01
75%      6.000000e+01
max      1.369000e+03
Name: new_listings, dtype: float64

count    6.293614e+06
mean     2.961404e+05
std      1.053433e+06
min      1.000000e+00
25%      1.330000e+05
50%      2.100000e+05
75%      3.415000e+05
max      1.000000e+09
Name: median_sale_price, dtype: float64

count    5.845714e+06
mean     3.206317e+05
std      2.069847e+06
min      1.000000e+02
25%      1.450000e+05
50%      2.274500e+05
75%      3.650000e+05
max      2.000000e+09
Name: median_list_price, dtype: float64


In [8]:
print(raw_data['inventory'].describe())

count    5.595925e+06
mean     4.327854e+01
std      6.730616e+01
min      1.000000e+00
25%      5.000000e+00
50%      1.700000e+01
75%      5.300000e+01
max      1.804000e+03
Name: inventory, dtype: float64


### create dataframe subset, df

In [5]:
#create subset df with important columns from above
df = raw_data[['period_begin', 'period_end', 'region', 
'state_code', 'property_type', 'homes_sold', 'new_listings', 
'median_sale_price', 'median_list_price', 'inventory']]

### change period_begin and period_end to datetime types

In [20]:
df2 = df.copy()

#period_begin
df2['period_begin'] = pd.to_datetime(df2['period_begin'], format="%Y/%m/%d").dt.date

In [21]:
#period_end
df2['period_end'] = pd.to_datetime(df2['period_end'], format="%Y/%m/%d").dt.date

# 1. What is the date range?
March 2012 - September 2022

In [22]:
#period begin
print(df2['period_begin'].describe(datetime_is_numeric = True))

#period end
print(df2['period_end'].describe(datetime_is_numeric = True))

count        6305970
unique           127
top       2021-06-01
freq           57722
Name: period_begin, dtype: object
count        6305970
unique           127
top       2021-08-31
freq           57722
Name: period_end, dtype: object


# 2. How many unique zip codes are there?
22,266 unique zip codes

### Extract Zip Codes

In [23]:
#extract zip code from region and create new series
df2['zip_code'] = df2['region'].str.replace('Zip Code: ', "")

In [14]:
#desribe zip codes
df2['zip_code'].describe()

count     6305970
unique      22266
top         02169
freq          635
Name: zip_code, dtype: object

In [24]:
display(df2.head(10))

,period_begin,period_end,region,state_code,property_type,homes_sold,new_listings,median_sale_price,median_list_price,inventory,zip_code
0,2021-08-01,2021-10-31,Zip Code: 48301,MI,All Residential,78.0,88.0,587000.0,639475.0,48.0,48301
1,2015-05-01,2015-07-31,Zip Code: 70076,LA,Single Family Residential,1.0,NaN,85000.0,NaN,1.0,70076
2,2018-08-01,2018-10-31,Zip Code: 14222,NY,Condo/Co-op,8.0,4.0,215000.0,199900.0,5.0,14222
3,2016-07-01,2016-09-30,Zip Code: 32966,FL,Condo/Co-op,40.0,39.0,132000.0,135900.0,44.0,32966
4,2020-09-01,2020-11-30,Zip Code: 39180,MS,All Residential,4.0,6.0,161950.0,190700.0,10.0,39180
5,2021-08-01,2021-10-31,Zip Code: 10465,NY,Condo/Co-op,12.0,27.0,407500.0,375000.0,31.0,10465
6,2020-05-01,2020-07-31,Zip Code: 60091,IL,Single Family Residential,104.0,166.0,826000.0,809000.0,102.0,60091
7,2018-03-01,2018-05-31,Zip Code: 94110,CA,Townhouse,3.0,5.0,1125000.0,1399000.0,3.0,94110
8,2017-05-01,2017-07-31,Zip Code: 84097,UT,All Residential,53.0,82.0,279000.0,310950.0,55.0,84097
9,2012-03-01,2012-05-31,Zip Code: 15074,PA,Single Family Residential,13.0,32.0,57700.0,77000.0,NaN,15074


In [18]:
#export dataframe to csv file
df2.to_csv (r'redfin_condensed.csv', index = False, header=True)

# 3. Create Nested Dictionary:
### {period_end: {zip codes: median_sale_price, ...}...}

In [14]:
# #extract unique period_end dates 
# df3 = df2.copy()
# period_end_list = df3['period_end'].values.tolist()
# unique_period_ends = list()
# for x in period_end_list:
#     if x not in unique_period_ends:
#         unique_period_ends.append(x)

In [15]:
# #extract zip code, period_end, and median_sale_price columns
# condensed_df = df3[['zip_code', 'period_end', 'median_sale_price']]

# #populate nested dict
# choropleth_dict = dict.fromkeys(unique_period_ends)

# for i in choropleth_dict:
#     choropleth_dict[i] = {}


# for index, row in condensed_df.iterrows():
#     date = row['period_end']
#     house_sale = row['median_sale_price']
#     zip_code = row['zip_code']
#     choropleth_dict[date][zip_code] = house_sale


# 4. Choropleth at Zip Code Level

## read in combined geo-json file for CA, WA, TX, and NC

In [23]:
file = "mergedfile.geojson"
with open("json_test_edits/{}".format(file)) as user_file:
    combined_geojson = json.load(user_file)


In [18]:
# #new york example
# df4 = df2.copy()
# df4_condensed = df4[['zip_code', 'period_end', 'median_sale_price', 'state_code']]

# #filter by new york and pick one end date
# df4_filtered = df4_condensed[(df4_condensed["state_code"] == "NY") & (df4_condensed["period_end"] == datetime.date(2021,10,31))]

# ny_data = df4_filtered[['zip_code', 'median_sale_price']]



# 2016 (wa, ca, tx, nc)

In [19]:
#combine alabama and alaska
df5 = df2.copy()
df5_condensed = df5[['zip_code', 'period_end', 'median_sale_price', 'state_code']]

#filter by ca
ca_filtered_2016 = df5_condensed[(df5_condensed["state_code"] == "CA") & (df5_condensed["period_end"] == datetime.date(2016,1,31))]

#filter by wa
wa_filtered_2016 = df5_condensed[(df5_condensed["state_code"] == "WA") & (df5_condensed["period_end"] == datetime.date(2016,1,31))]

#filter by TX
tx_filtered_2016 = df5_condensed[(df5_condensed["state_code"] == "TX") & (df5_condensed["period_end"] == datetime.date(2016,1,31))]

#filter by wa
nc_filtered_2016 = df5_condensed[(df5_condensed["state_code"] == "NC") & (df5_condensed["period_end"] == datetime.date(2016,1,31))]

#merge
ca_data_2016 = ca_filtered_2016[['zip_code', 'median_sale_price']]
wa_data_2016 = wa_filtered_2016[['zip_code', 'median_sale_price']]
tx_data_2016 = tx_filtered_2016[['zip_code', 'median_sale_price']]
nc_data_2016 = nc_filtered_2016[['zip_code', 'median_sale_price']]

combined_data_2016 = pd.concat([ca_data_2016, wa_data_2016, tx_data_2016, nc_data_2016])

max_sale_2016 = combined_data_2016['median_sale_price'].max()
min_sale_2016 = combined_data_2016['median_sale_price'].min()

print(max_sale_2016, min_sale_2016)

12500000.0 7400.0


# 2022 (wa, ca, tx, nc)

In [22]:
#combine alabama and alaska
df5 = df2.copy()
df5_condensed = df5[['zip_code', 'period_end', 'median_sale_price', 'state_code']]

#filter by ca
ca_filtered_2022 = df5_condensed[(df5_condensed["state_code"] == "CA") & (df5_condensed["period_end"] == datetime.date(2022,9,30))]

#filter by wa
wa_filtered_2022 = df5_condensed[(df5_condensed["state_code"] == "WA") & (df5_condensed["period_end"] == datetime.date(2022,9,30))]

#filter by TX
tx_filtered_2022 = df5_condensed[(df5_condensed["state_code"] == "TX") & (df5_condensed["period_end"] == datetime.date(2022,9,30))]

#filter by wa
nc_filtered_2022 = df5_condensed[(df5_condensed["state_code"] == "NC") & (df5_condensed["period_end"] == datetime.date(2022,9,30))]

#merge
ca_data_2022 = ca_filtered_2022[['zip_code', 'median_sale_price']]
wa_data_2022 = wa_filtered_2022[['zip_code', 'median_sale_price']]
tx_data_2022 = tx_filtered_2022[['zip_code', 'median_sale_price']]
nc_data_2022 = nc_filtered_2022[['zip_code', 'median_sale_price']]

combined_data_2022 = pd.concat([ca_data_2022, wa_data_2022, tx_data_2022, nc_data_2022])

max_sale_2022 = combined_data_2022['median_sale_price'].max()
min_sale_2022 = combined_data_2022['median_sale_price'].min()

print(max_sale_2022, min_sale_2022)

12500000.0 1.0


In [24]:
fig = px.choropleth(combined_data_2016, geojson = combined_geojson, locations = 'zip_code', color = 'median_sale_price',
                                    featureidkey = "properties.ZCTA5CE10",
                                    color_continuous_scale="Viridis",
                                    range_color=(min_sale_2016, max_sale_2022),
                                    scope="usa",
                                    labels={'median_sale_price':'median sale price'}
                                    )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
fig1 = px.choropleth(combined_data_2022, geojson = combined_geojson, locations = 'zip_code', color = 'median_sale_price',
                                    featureidkey = "properties.ZCTA5CE10",
                                    color_continuous_scale="Viridis",
                                    range_color=(min_sale_2016, max_sale_2022),
                                    scope="usa",
                                    labels={'median_sale_price':'median sale price'}
                                    )

fig1.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig1.show()

In [ ]:
#hellow